In [1]:
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load       Tokens  Owns (effective)  Host ID                               Rack 
UN  172.25.0.2  70.26 KiB  16      74.1%             5ddcc947-9ed4-4b55-87c1-8164a211ea4b  rack1
UN  172.25.0.4  99.42 KiB  16      68.2%             d553f1d9-458c-41fc-b429-3728e6e218f9  rack1
UN  172.25.0.3  70.25 KiB  16      57.8%             f640d31c-a0bf-4f22-96d0-1946135d9d71  rack1



In [2]:
from cassandra.cluster import Cluster
cluster = Cluster(['p6-db-1', 'p6-db-2', 'p6-db-3'])
cass = cluster.connect()

In [3]:
cass.execute("drop keyspace if exists weather")
cass.execute("create keyspace weather with replication={'class': 'SimpleStrategy', 'replication_factor': 3};")
cass.execute("create type if not exists weather.station_record(tmin INT, tmax INT)")
cass.execute('''create table if not exists weather.stations (
id TEXT, 
name TEXT STATIC, 
date DATE, 
record weather.station_record,
PRIMARY KEY ((id), date))
WITH CLUSTERING ORDER BY (date ASC)''')

In [4]:
#q1

cass.execute("describe table weather.stations").one().create_statement

"CREATE TABLE weather.stations (\n    id text,\n    date date,\n    name text static,\n    record station_record,\n    PRIMARY KEY (id, date)\n) WITH CLUSTERING ORDER BY (date ASC)\n    AND additional_write_policy = '99p'\n    AND bloom_filter_fp_chance = 0.01\n    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}\n    AND cdc = false\n    AND comment = ''\n    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}\n    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}\n    AND memtable = 'default'\n    AND crc_check_chance = 1.0\n    AND default_time_to_live = 0\n    AND extensions = {}\n    AND gc_grace_seconds = 864000\n    AND max_index_interval = 2048\n    AND memtable_flush_period_in_ms = 0\n    AND min_index_interval = 128\n    AND read_repair = 'BLOCKING'\n    AND speculative_retry = '99p';"

In [5]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("p6")
         .config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.0')
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .getOrCreate())

:: loading settings :: url = jar:file:/spark-3.4.1-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9fe4a25a-ff29-49da-ad44-d9c207c0fa09;1.0
	confs: [default]


	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 in central


	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central


	found com.typesafe#config;1.4.1 in central


...

	[SUCCESSFUL ] org.scala-lang#scala-reflect;2.12.11!scala-reflect.jar (145ms)
downloading https://repo1.maven.org/maven2/com/datastax/oss/native-protocol/1.5.0/native-protocol-1.5.0.jar ...
	[SUCCESSFUL ] com.datastax.oss#native-protocol;1.5.0!native-protocol.jar(bundle) (51ms)
downloading https://repo1.maven.org/maven2/com/datastax/oss/java-driver-shaded-guava/25.1-jre-graal-sub-1/java-driver-shaded-guava-25.1-jre-graal-sub-1.jar ...
	[SUCCESSFUL ] com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1!java-driver-shaded-guava.jar (126ms)
downloading https://repo1.maven.org/maven2/com/typesafe/config/1.4.1/config-1.4.1.jar ...


	[SUCCESSFUL ] com.typesafe#config;1.4.1!config.jar(bundle) (30ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.26/slf4j-api-1.7.26.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.26!slf4j-api.jar (27ms)
downloading https://repo1.maven.org/maven2/io/dropwizard/metrics/metrics-core/4.1.18/metrics-core-4.1.18.jar ...
	[SUCCESSFUL ] io.dropwizard.metrics#metrics-core;4.1.18!metrics-core.jar(bundle) (30ms)
downloading https://repo1.maven.org/maven2/org/hdrhistogram/HdrHistogram/2.1.12/HdrHistogram-2.1.12.jar ...
	[SUCCESSFUL ] org.hdrhistogram#HdrHistogram;2.1.12!HdrHistogram.jar(bundle) (31ms)
downloading https://repo1.maven.org/maven2/org/reactivestreams/reactive-streams/1.0.3/reactive-streams-1.0.3.jar ...
	[SUCCESSFUL ] org.reactivestreams#reactive-streams;1.0.3!reactive-streams.jar (26ms)
downloading https://repo1.maven.org/maven2/com/github/stephenc/jcip/jcip-annotations/1.0-1/jcip-annotations-1.0-1.jar ...
	[SUCCESSFUL ] com.github.stephenc.jcip#jcip-annotations

	[SUCCESSFUL ] com.google.code.findbugs#jsr305;3.0.2!jsr305.jar (29ms)
downloading https://repo1.maven.org/maven2/com/datastax/oss/java-driver-query-builder/4.13.0/java-driver-query-builder-4.13.0.jar ...
	[SUCCESSFUL ] com.datastax.oss#java-driver-query-builder;4.13.0!java-driver-query-builder.jar(bundle) (31ms)
:: resolution report :: resolve 4745ms :: artifacts dl 1329ms
	:: modules in use:
	com.datastax.oss#java-driver-core-shaded;4.13.0 from central in [default]
	com.datastax.oss#java-driver-mapper-runtime;4.13.0 from central in [default]
	com.datastax.oss#java-driver-query-builder;4.13.0 from central in [default]
	com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 from central in [default]
	com.datastax.oss#native-protocol;1.5.0 from central in [default]
	com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 from central in [default]
	com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 from central in [default]
	com.github.spotbugs#spotbugs-annotations;3.

23/11/23 02:02:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
df = spark.read.format("text").load("./ghcnd-stations.txt")

In [7]:
from pyspark.sql.functions import substring
import pandas as pd

df_new = df.withColumn("ID", substring("value", 1, 11)).withColumn("STATE", substring("value", 39, 2)) \
    .withColumn("NAME", substring("value", 42, 30)).drop("value")
df_new = df_new[df_new['STATE']=='WI']
print(df_new)

cass.execute("truncate table weather.stations")

insert_stations = cass.prepare("INSERT INTO weather.stations (id, name) VALUES (?, ?)")
for row in df_new.collect():
    id_, name_ = row['ID'], row['NAME']
    cass.execute(insert_stations, (id_, name_))
# pd.DataFrame(cass.execute("select count(*) from weather.stations"))

DataFrame[ID: string, STATE: string, NAME: string]


In [8]:
import pandas as pd
pd.DataFrame(cass.execute("select * from weather.stations"))

,id,date,name,record
0,USC00479053,None,W BEND FIRE STN #2,None
1,USC00476398,None,PARK FALLS DNR HQ,None
2,USC00470268,None,APPOLONIA,None
3,USC00474110,None,JUNEAU,None
4,USC00475525,None,MINONG 5 WSW,None
...,...,...,...,...
1308,USC00479050,None,WEST BEND,None
1309,US1WIPC0004,None,RIVER FALLS 3.0 SE,None
1310,US1WIWK0016,None,MUSKEGO 1.0 W,None
1311,US1WIWK0086,None,WALES 0.4 NW,None


In [9]:
cass.execute("SELECT COUNT(*) FROM weather.stations").all() # 1313

[Row(count=1313)]

In [10]:
#q2
pd.DataFrame(cass.execute("""SELECT name FROM weather.stations WHERE id = 'USW00014837'"""))['name']

0    MADISON DANE CO RGNL AP       
Name: name, dtype: object

In [11]:
#q3

token = pd.DataFrame(cass.execute("SELECT token(id) FROM weather.stations WHERE id = 'USC00470273'"))['system_token_id'][0]
token

-9014250178872933741

In [12]:
#q4

import subprocess
import re

output = subprocess.check_output(["nodetool", "ring"], universal_newlines=True)
lines = output.split('\n')
# print(lines[5])
vnode_tokens = []
for l in range(len(lines)):
    if l<=3:
        continue
    if lines[l]=='':
        break
    vnode_tokens.append(int(lines[l][80:110]))
vnode_tokens.sort()

vnode = None
for vnode_token in vnode_tokens:
    # print(vnode_token, token)
    if vnode_token >= token:
        # print(True)
        vnode = vnode_token
        break
if vnode == None:
    vnode = vnode_tokens[0]

vnode

-8897995659956756393

In [13]:
!rm -rf records.parquet
!apt install unzip
!unzip records.zip 



unzip is already the newest version (6.0-26ubuntu3.1).


0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.


Archive:  records.zip
   creating: records.parquet/
  inflating: records.parquet/part-00000-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet  
 extracting: records.parquet/._SUCCESS.crc  
  inflating: records.parquet/part-00002-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet  
  inflating: records.parquet/part-00001-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet  
  inflating: records.parquet/part-00003-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet  
 extracting: records.parquet/.part-00003-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet.crc  
 extracting: records.parquet/_SUCCESS  
 extracting: records.parquet/.part-00000-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet.crc  
 extracting: records.parquet/.part-00001-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet.crc  
 extracting: records.parquet/.part-00002-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet.crc  


In [14]:
!python3 -m grpc_tools.protoc -I=. --python_out=. --grpc_python_out=. station.proto

In [15]:
records_df = spark.read.parquet("records.parquet")

records_df.show()

+-----------+--------+-------+------+
|    station|    date|element| value|
+-----------+--------+-------+------+
|USW00014898|20220101|   TMAX| -32.0|
|USW00014898|20220102|   TMAX| -77.0|
|USW00014898|20220103|   TMAX| -60.0|
|USW00014898|20220104|   TMAX|   0.0|
|USW00014898|20220105|   TMAX| -16.0|
|USW00014898|20220106|   TMAX| -71.0|
|USW00014898|20220107|   TMAX| -71.0|
|USW00014898|20220108|   TMAX| -32.0|
|USW00014898|20220109|   TMAX| -27.0|
|USW00014898|20220110|   TMAX|-149.0|
|USW00014898|20220111|   TMAX| -16.0|
|USW00014898|20220112|   TMAX|   6.0|
|USW00014898|20220113|   TMAX|  11.0|
|USW00014898|20220114|   TMAX| -77.0|
|USW00014898|20220115|   TMAX| -99.0|
|USW00014898|20220116|   TMAX| -60.0|
|USW00014898|20220117|   TMAX| -21.0|
|USW00014898|20220118|   TMAX|  28.0|
|USW00014898|20220119|   TMAX|  28.0|
|USW00014898|20220120|   TMAX|-121.0|
+-----------+--------+-------+------+
only showing top 20 rows



In [16]:
records_df = records_df.groupBy("station", "date").pivot("element", ["TMAX", "TMIN"]).sum("value")
records_df.show()

+-----------+--------+------+------+
|    station|    date|  TMAX|  TMIN|
+-----------+--------+------+------+
|USW00014898|20220107| -71.0|-166.0|
|USW00014839|20220924| 194.0| 117.0|
|USW00014839|20220523| 150.0|  83.0|
|USW00014839|20221019|  83.0|  11.0|
|USW00014839|20220529| 261.0| 139.0|
|USR0000WDDG|20221130| -39.0|-106.0|
|USR0000WDDG|20220119| -56.0|-178.0|
|USW00014837|20220222| -38.0| -88.0|
|USR0000WDDG|20220202|-106.0|-150.0|
|USW00014839|20220427|  39.0|   0.0|
|USW00014839|20220708| 222.0| 189.0|
|USW00014839|20220917| 294.0| 200.0|
|USW00014837|20220624| 322.0| 200.0|
|USW00014898|20220129| -60.0|-116.0|
|USW00014839|20220715| 233.0| 156.0|
|USR0000WDDG|20220224| -61.0|-128.0|
|USR0000WDDG|20220130| -33.0|-117.0|
|USR0000WDDG|20220414|  50.0| -17.0|
|USW00014898|20220728| 256.0| 156.0|
|USW00014837|20220906| 256.0| 117.0|
+-----------+--------+------+------+
only showing top 20 rows



In [17]:
records_df.toPandas()

,station,date,TMAX,TMIN
0,USW00014898,20220107,-71.0,-166.0
1,USW00014839,20220924,194.0,117.0
2,USW00014839,20220523,150.0,83.0
3,USW00014839,20221019,83.0,11.0
4,USW00014839,20220529,261.0,139.0
...,...,...,...,...
1455,USW00014898,20220724,278.0,167.0
1456,USW00014837,20221004,222.0,50.0
1457,USW00014837,20221107,94.0,17.0
1458,USW00014898,20221006,200.0,56.0


In [18]:
#q5
# loop over the results, making a call to the server with for each row to insert the measurements to the database
import grpc
import station_pb2_grpc, station_pb2

channel = grpc.insecure_channel('localhost:5440')
stub = station_pb2_grpc.StationStub(channel)

for row in records_df.collect():
    resp = stub.RecordTemps(station_pb2.RecordTempsRequest(station=row.station, date=row.date, 
                                                            tmin=int(row.TMIN), tmax=int(row.TMAX)))

resp = stub.StationMax(station_pb2.StationMaxRequest(station='USW00014837'))
# print(resp)

resp.tmax

356

In [19]:
df = spark.read.format("org.apache.spark.sql.cassandra")\
.option("spark.cassandra.connection.host", "p6-db-1,p6-db-2,p6-db-3")\
.option("keyspace", "weather")\
.option("table", "stations")\
.load()

In [20]:
spark.conf.set("spark.sql.catalog.mycat", "com.datastax.spark.connector.datasource.CassandraCatalog")
spark.conf.set("spark.sql.catalog.mycat.spark.cassandra.connection.host", "p6-db-1,p6-db-2,p6-db-3")

In [21]:
df.createOrReplaceTempView("stations")

In [22]:
#q6 : List tables and views in Spark Catalog
spark.catalog.listTables()

[Table(name='stations', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [23]:
#q7 : what is the average difference between tmax and tmin, for each of the four stations that have temperature records?

df_res = df.filter("record is not null")
df_res = df_res.withColumn('difference', ( df['record.tmax'] - df['record.tmin'] ))
df_res = df_res.groupBy('id').agg({'difference': 'mean'})
df_res.collect()
d = {}
for row in df_res.collect():
    rd = row.asDict()
    d[rd["id"]] = rd["avg(difference)"]

d
# df_
    # map(lambda row: row.asDict(), df.collect()))

{'USW00014839': 89.6986301369863,
 'USW00014898': 102.93698630136986,
 'USR0000WDDG': 102.06849315068493,
 'USW00014837': 105.62739726027397}

In [24]:
#q8

!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load        Tokens  Owns (effective)  Host ID                               Rack 
UN  172.25.0.2  118.22 KiB  16      100.0%            5ddcc947-9ed4-4b55-87c1-8164a211ea4b  rack1
UN  172.25.0.4  92.8 KiB    16      100.0%            d553f1d9-458c-41fc-b429-3728e6e218f9  rack1
DN  172.25.0.3  92.8 KiB    16      100.0%            f640d31c-a0bf-4f22-96d0-1946135d9d71  rack1



In [25]:
#q9

resp = stub.StationMax(station_pb2.StationMaxRequest(station='USW00014837'))

resp.error

'need 3 replicas, but only have 2'

In [26]:
#q10

resp = stub.RecordTemps(station_pb2.RecordTempsRequest(station='USW00019837', date='21011120', 
                                                            tmin=0, tmax=0))

resp.error

''